[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_add_stamp.ipynb)

In [ ]:
%pip install pdftools_toolbox
%pip install ipython

# Add stamp to PDF
Add a semi-transparent stamp text onto each page of a PDF
document. Optionally specify the color and the opacity of
the stamp.

In [ ]:
import io
import math
from pdftools_toolbox.pdf import Document, FileReference, Metadata, Page, PageCopyOptions
from pdftools_toolbox.geometry.real import AffineTransform, Point
from pdftools_toolbox.pdf.content import ColorSpace, ContentGenerator, Font, IccBasedColorSpace, Paint, ProcessColorSpaceType, Text, TextGenerator, Transparency
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_file_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_file_path)
stamp_string = "APPROVED"
output_file_path = 'InvoiceWithStamp.pdf'
alpha = 0.5

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Metadata
    out_doc.metadata = Metadata.copy(out_doc, in_doc.metadata)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
def add_stamp(output_doc: Document, out_page: Page, stamp_string: str):
    # Create content generator and text object
    with ContentGenerator(out_page.content, False) as gen:
        text = Text.create(output_doc)

        # Create text generator
        with TextGenerator(text, font, font_size, None) as text_generator:
            # Calculate point and angle of rotation
            rotation_center = Point(
                x=out_page.size.width / 2.0,
                y=out_page.size.height / 2.0
            )
            rotation_angle = math.atan2(out_page.size.height, out_page.size.width) / math.pi * 180.0

            # Rotate text input around the calculated position
            trans = AffineTransform.get_identity()
            trans.rotate(rotation_angle, rotation_center)
            gen.transform(trans)

            # Calculate position
            position = Point(
                x=(out_page.size.width - text_generator.get_width(stamp_string)) / 2.0,
                y=(out_page.size.height - font.ascent * font_size) / 2.0
            )

            # Move to position
            text_generator.move_to(position)

            # Set text paint
            text_generator.fill = paint

            # Add given stamp string
            text_generator.show_line(stamp_string)

        # Paint the positioned text
        gen.paint_text(text)

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Define global variables
    font_size = 50.0
    
    # Open input document
    with io.FileIO(input_file_path, 'rb') as in_stream:
        with Document.open(in_stream, None) as in_doc:
    
            # Create output document
            with io.FileIO(output_file_path, 'wb+') as output_stream:
                with Document.create(output_stream, in_doc.conformance, None) as out_doc:
    
                    # Copy document-wide data
                    copy_document_data(in_doc, out_doc)
    
                    # Create font
                    font = Font.create_from_system(out_doc, "Arial", "Italic", True)
    
                    # Get the color space
                    color_space = ColorSpace.create_process_color_space(out_doc, ProcessColorSpaceType.RGB)
    
                    # Choose the RGB color value and transparency
                    color = [1.0, 0.0, 0.0]
                    transparency = Transparency(alpha)
    
                    # Create paint object with the chosen RGB color
                    paint = Paint.create(out_doc, color_space, color, transparency)
    
                    # Define copy options
                    copy_options = PageCopyOptions()
    
                    # Copy all pages from input document
                    for in_page in in_doc.pages:
                        # Copy page from input to output
                        out_page = Page.copy(out_doc, in_page, copy_options)
    
                        # Add text to page
                        add_stamp(out_doc, out_page, stamp_string)
    
                        # Add page to document
                        out_doc.pages.append(out_page)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")